In [ ]:
# --- Install Libraries ---
!pip install pandas gcsfs pyarrow scikit-learn xgboost sentence-transformers -q

# --- Import Libraries ---
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import os

# --- Load the Fine-Tuned Model from GCS bucket ---
bucketName = 'wanderlust-recommender-system'
modelPath = f'gs://{bucketName}/processed/hotel_recommender_finetuned/'
localModelPath = 'finetunedModel'

# --- Download the Model from GCS to the Local Instance ---
# Create the local directory if it doesn't exist
if not os.path.exists(localModelPath):
    os.makedirs(localModelPath)

# Use gsutil to copy the files
print("Downloading fine-tuned model from GCS bucket ...")
!gsutil -m cp -r {modelPath}* {localModelPath}
print("Model downloaded successfully.")

print("Loading the fine-tuned model ...")
model = SentenceTransformer(localModelPath)
print("Model loaded successfully.")

# --- Load the Cleaned Hotel Review Data ---
dataPath = f'gs://{bucketName}/processed/combined_hotel_reviews.parquet'
print("Cleaned data loading ...")
df = pd.read_parquet(dataPath)
print("Data Loaded successfully.")

In [ ]:
# --- Consolidate Reviews for Each Hotel ---
print("Consolidating all reviews for each hotel ...")
dfConsolidated = df.groupby('hotel_id')['reviews.text'].\
apply(lambda x: ' '.join(x)).reset_index()
dfConsolidated.rename(columns={'reviews.text': 'allReviews'}, inplace=True)
print(f'Consolidation done successfully for {len(dfConsolidated)} hotels.')

# --- Generate Embeddings ---
embeddedData = model.encode(dfConsolidated.allReviews.tolist(),\
                            show_progress_bar=True)

# --- Create a Hotel ID to Embedding Dictionary ---
embeddingDict = dict(zip(dfConsolidated.hotel_id, embeddedData))
print(f'Embedding dictionary created for {len(embeddedData)} hotels.')
print('Saving numpy object for new embeddings ...')
localPathEmbedding = 'newEmbedding.npy'
np.save(localPathEmbedding,embeddingDict)
print('Object saved on local instance.')


# We check the shape of one embedding to see its dimensionality.
print(f'Check for dimension of an input in embedding file: {next(iter(embeddingDict.values())).shape}')

# --- Save Embeddings to GCS ---
print('Saving embedding object to GCS bucket ...')
!gsutil -m cp {localPathEmbedding} gs://{bucketName}/processed/
print('Saved successfully.')

In [ ]:
df['reviews.rating']

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

# --- Preprocess for Uniqueness ---
# Group by user and hotel to find the average rating for any user for a single hotel
print('Check for duplicate ratings for each hotel and then averaging ...')
uniquePairs = df.groupby(['user_id','hotel_id'])['reviews.rating'].mean().reset_index()
print('Unique pairs of user_id and hotel_id created.')

# --- Create the User-Item Interaction Matrix ---
nuniqueUsers = uniquePairs.user_id.nunique()
nuniqueHotels = uniquePairs.hotel_id.nunique()
print('Creating user-hotel interaction matrix ...')
interactionMatrix = csr_matrix((uniquePairs['reviews.rating'],(uniquePairs.user_id,uniquePairs.hotel_id)), shape=(nuniqueUsers,nuniqueHotels))
print('Matrix created.')

# --- Decompose the Matrix with TruncatedSVD ---
print('SVD computation started with 50 latant features ...')
svd = TruncatedSVD(n_components=50, random_state=42)
svd.fit(interactionMatrix)
print('SVD computation done.')



# --- Extract User and Hotel Factors ---
print('Extracting user and hotel factors matrices ...')
hotelFactor = svd.components_.T
userFactor = svd.transform(interactionMatrix)
print(f'User and hotel fators matrices created with {userFactor.shape} and {hotelFactor.shape} shapes respectively.')


# --- Save SVD Factors to GCS ---
userPath, hotelPath = 'userFactor.npy', 'hotelFactor.npy'
np.save(userPath, userFactor)
np.save(hotelPath, hotelFactor)
print('Saving user and hotel factors matrices on GCS bucket ...')
!gsutil -m cp userFactor.npy gs://{bucketName}/processed/
!gsutil -m cp hotelFactor.npy gs://{bucketName}/processed/
!rm -f {userPath} {hotelPath}
print('Saved successfully.')